In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
from scipy.stats import ttest_rel, sem

In [ ]:
%load_ext autoreload
%autoreload 2
plt.style.use('bayesee.academic')

In [ ]:
repo_path = Path.cwd().parents[0]
print(repo_path)

In [ ]:
subject = "AZ_t1"
p3_spatial_statistics = pd.read_csv(
    repo_path / f"data/covert-search/large-field/p3_spatial_statistics_{subject}.csv"
)
p4_spatial_statistics = pd.read_csv(
    repo_path / f"data/covert-search/large-field/p4_spatial_statistics_{subject}.csv"
)

In [ ]:
indexes_near = p4_spatial_statistics["id"].values

In [ ]:
large_field_near_region_hit = p3_spatial_statistics.loc[indexes_near - 1, "hit_rate"]
large_field_near_region_miss = p3_spatial_statistics.loc[indexes_near - 1, "miss_rate"]
large_field_near_region_fa = p3_spatial_statistics.loc[indexes_near - 1, "fa_rate"]

In [ ]:
small_field_hit = p4_spatial_statistics["hit_rate"]
small_field_miss = p4_spatial_statistics["miss_rate"]
small_field_fa = p4_spatial_statistics["fa_rate"]

In [ ]:
n_location = len(p3_spatial_statistics) + 1

orientation_hsv_color_map = np.zeros((n_location - 1, 3))
orientation_hsv_color_map[:, 0] = p3_spatial_statistics["orientation"]

for index_location in range(n_location - 1):
    orientation_hsv_color_map[index_location, 1] = 1

    if index_location == 0:
        orientation_hsv_color_map[index_location, 2] = 0
    else:
        orientation_hsv_color_map[index_location, 2] = 0.75

rgb_orientation_color_map = hsv_to_rgb(orientation_hsv_color_map)

In [ ]:
rgb_orientation_color_map[indexes_near - 1, :].shape

In [ ]:
for paired_statistics in [
    (large_field_near_region_hit, small_field_hit),
    (large_field_near_region_miss, small_field_miss),
    (large_field_near_region_fa, small_field_fa),
]:
    before, after = paired_statistics

    fig, ax = plt.subplots()

    for index_pair, point_before in enumerate(before):
        ax.plot(
            [0, 1],
            [point_before, after[index_pair]],
            "-",
            linewidth=3,
            color=rgb_orientation_color_map[indexes_near[index_pair] - 1, :],
            alpha=0.75,
        )

    ax.errorbar(
        [0, 1],
        [before.mean(), after.mean()],
        yerr=[sem(before), sem(after)],
        color="black",
        linewidth=7,
    )

    ax.set(ylabel="Hit", xticks=[0, 1])
    ax.set_xticklabels(["16-degree field", "8-degree field"], fontsize=24)

    plt.show()

    t_val, p_val = ttest_rel(after, before)
    print(
        f"Mean difference: {(after-before).mean():.3f}; (paired sample t-test) t-statistic: {t_val:.3f}, p-value: {p_val:.5f}."
    )